本章讨论几个重要的魔法方法，其中最重要的是__init__以及一些处理元素访问的方法（它们让你能够创建序列或映射）。本章还将讨论两个相关的主题：特性（property）和迭代器（iterator）。前者以前是通过魔法方法处理的，但现在通过函数property处理，而后者使用魔法方法__iter__，这让其可用于for循环中。在本章最后，将通过一个内容丰富的示例演示如何使用已有知识来解决非常棘手的问题。

9.2 构造函数我们要介绍的第一个魔法方法是构造函数。你可能从未听说过构造函数（constructor），它其实就是本书前面一些示例中使用的初始化方法，只是命名为__init__。然而，构造函数不同于普通方法的地方在于，将在对象创建后自动调用它们

In [4]:
        class FooBar:
            def __init__(self):
              self.somevar = 42
            
            def __init__(self, value=42):
              self.somevar = value





In [5]:
        >>> f = FooBar()
        >>> f.somevar

42

In [6]:
        >>> f = FooBar('This is a constructor argument')
        >>> f.somevar


'This is a constructor argument'

9.2.1 重写普通方法和特殊的构造函数第7章介绍了继承。每个类都有一个或多个超类，并从它们那里继承行为。对类B的实例调用方法（或访问其属性）时，如果找不到该方法（或属性），将在其超类A中查找。请看下面两个类：

In [7]:
        class A:
            def hello(self):
              print("Hello, I'm A.")


        class B(A):
            pass

In [8]:
        >>> a = A()
        >>> b = B()
        >>> a.hello()
        >>> b.hello()

Hello, I'm A.
Hello, I'm A.


In [9]:
        class B(A):
            def hello(self):
              print("Hello, I'm B.")

In [10]:
        >>> b = B()
        >>> b.hello()

Hello, I'm B.


虽然所有方法的重写机制都相同，但与重写普通方法相比，重写构造函数时更有可能遇到一个特别的问题：重写构造函数时，必须调用超类（继承的类）的构造函数，否则可能无法正确地初始化对象。请看下面的Bird类：

In [13]:
        class Bird:
            def __init__(self):
              self.hungry = True
            def eat(self):
                if self.hungry:
                    print('Aaaah ...')
                    self.hungry = False
                else:
                    print('No, thanks!')

这个类定义了所有鸟都具备的一种基本能力：进食。下面的示例演示了如何使用这个类：

In [14]:
        >>> b = Bird()
        >>> b.eat()


Aaaah ...


In [15]:
 >>> b.eat()

No, thanks!


从这个示例可知，鸟进食后就不再饥饿。下面来看子类SongBird，它新增了鸣叫功能。

In [16]:
        class SongBird(Bird):
            def __init__(self):
              self.sound = 'Squawk!'
            def sing(self):
              print(self.sound)

In [17]:
        >>> sb = SongBird()
        >>> sb.sing()

Squawk!


In [18]:
sb.eat()

AttributeError: 'SongBird' object has no attribute 'hungry'

异常清楚地指出了问题出在什么地方：SongBird没有属性hungry。为何会这样呢？因为在SongBird中重写了构造函数，但新的构造函数没有包含任何初始化属性hungry的代码。要消除这种错误，SongBird的构造函数必须调用其超类（Bird）的构造函数，以确保基本的初始化得以执行。为此，有两种方法：调用未关联的超类构造函数，以及使用函数super。接下来的两节将介绍这两种方法。

9.2.2 调用未关联的超类构造函数

In [24]:
        class SongBird(Bird):
            def __init__(self):
                 Bird.__init__(self)   #调用父类的初始化函数
                 self.sound = 'Squawk!'
            def sing(self):
                print(self.sound)

In [25]:
        >>> sb = SongBird()
        >>> sb.sing()
      

Squawk!


In [21]:
sb.eat()

Aaaah ...


In [22]:
sb.eat()

No, thanks!


In [23]:
sb.eat()

No, thanks!


9.2.3 使用函数super

如果你使用的不是旧版Python，就应使用函数super。这个函数只适用于新式类，而你无论如何都应使用新式类。调用这个函数时，将当前类和当前实例作为参数。对其返回的对象调用方法时，调用的将是超类（而不是当前类）的方法。因此，在SongBird的构造函数中，可不使用Bird，而是使用super(SongBird, self)。另外，可像通常那样（也就是像调用关联的方法那样）调用方法__init__。在Python 3中调用函数super时，可不提供任何参数（通常也应该这样做），而它将像变魔术一样完成任务。

In [26]:
        class Bird:
            def __init__(self):
              self.hungry = True
            def eat(self):
              if self.hungry:
                  print('Aaaah ...')
                  self.hungry = False
              else:
                  print('No, thanks!')


        class SongBird(Bird):
            def __init__(self):
                super().__init__()
                self.sound = 'Squawk!'
            def sing(self):
                print(self.sound)

In [27]:
        >>> sb = SongBird()
        >>> sb.sing()
        >>> sb.eat()



Squawk!
Aaaah ...


In [28]:
       >>> sb.eat()

No, thanks!


9.3 元素访问

9.3.1 基本的序列和映射协议序列和映射基本上是元素（item）的集合，要实现它们的基本行为（协议），不可变对象需要实现2个方法，而可变对象需要实现4个。

❑ __len__(self)：这个方法应返回集合包含的项数，对序列来说为元素个数，对映射来说为键-值对数。如果__len__返回零（且没有实现覆盖这种行为的__nonzero__），对象在布尔上下文中将被视为假（就像空的列表、元组、字符串和字典一样）。
❑ __getitem__(self, key)：这个方法应返回与指定键相关联的值。对序列来说，键应该是0～n-1的整数（也可以是负数，这将在后面说明），其中n为序列的长度。对映射来说，键可以是任何类型。
❑ __setitem__(self, key, value)：这个方法应以与键相关联的方式存储值，以便以后能够使用__getitem__来获取。当然，仅当对象可变时才需要实现这个方法。
❑ __delitem__(self, key)：这个方法在对对象的组成部分使用__del__语句时被调用，应删除与key相关联的值。同样，仅当对象可变（且允许其项被删除）时，才需要实现这个方法。

对于这些方法，还有一些额外的要求。
❑ 对于序列，如果键为负整数，应从末尾往前数。换而言之，x[-n]应与x[len(x)-n]等效。
❑ 如果键的类型不合适（如对序列使用字符串键），可能引发TypeError异常。
❑ 对于序列，如果索引的类型是正确的，但不在允许的范围内，应引发IndexError异常。

In [46]:
        def check_index(key):
            """
            指定的键是否是可接受的索引？
            键必须是非负整数，才是可接受的。如果不是整数，
            将引发TypeError异常；如果是负数，将引发Index
            Error异常（因为这个序列的长度是无穷的）
            """
            if not isinstance(key, int):
                raise TypeError
            if  key < 0:
                raise IndexError

In [65]:

        class ArithmeticSequence:
            def __init__(self, start=0, step=1):
                self.start = start                                # 存储起始值
                self.step = step                                  # 存储步长值
                self.changed = {}                                  # 没有任何元素被修改
            
            def __setitem__(self, key, value):
                check_index(key)
                self.changed[key] = value                       # 存储修改后的值


            def __getitem__(self, key):
                check_index(key)
                try:
                    return self.changed[key]                    # 修改过？
                except KeyError:                                  # 如果没有修改过，
                    return self.start + key * self.step          # 就计算元素的值


                

In [67]:
        >>> s = ArithmeticSequence(1, 2)
        >>> s[4]
      #读取序列值，实际会调用内部函数__getitem__

9

In [68]:
s[4]=2
s[4]

2

In [69]:
s[5]

11

In [70]:
s[1]

3

In [72]:
s[7]

15

In [75]:
s[1000]

2001

9.3.2 从list、dict和str派生

基本的序列/映射协议指定的4个方法能够让你走很远，但序列还有很多其他有用的魔法方法和普通方法，其中包括将在9.6节介绍的方法__iter__。要实

那么该如何做呢？“咒语”就是继承。在能够继承的情况下为何去重新实现呢？在标准库中，模块collections提供了抽象和具体的基类，但你也可以继承内置类型。因此，如果要实现一种行为类似于内置列表的序列类型，可直接继承list.来看一个简单的示例——一个带访问计数器的列表。

In [77]:
        class CounterList(list):
            def __init__(self, *args):
                super().__init__(*args)
                self.counter = 0
            def __getitem__(self, index):
                self.counter += 1
                return super(CounterList, self).__getitem__(index)

In [78]:
        >>> cl = CounterList(range(10))
        >>> cl
    
     

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [79]:
>>> cl.reverse()
>>> cl

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [85]:
cl.counter

2

In [84]:
cl[1:3]

[8, 7]

In [86]:
        class Rectangle:
            def __init__(self):
              self.width = 0
              self.height = 0
            def set_size(self, size):
              self.width, self.height = size
            def get_size(self):
              return self.width, self.height

In [88]:
        >>> r = Rectangle()
        >>> r.width = 10
        >>> r.height = 5
        >>> r.get_size()
       

(10, 5)

In [89]:

        >>> r.set_size((150, 100))
        >>> r.width


150

get_size和set_size是假想属性size的存取方法，这个属性是一个由width和height组成的元组。（可随便将这个属性替换为更有趣的属性，如矩形的面积或其对角线长度。）这些代码并非完全错误，但存在缺陷。使用这个类时，程序员应无需关心它是如何实现的（封装）。如果有一天你想修改实现，让size成为真正的属性，而width和height是动态计算出来的，就需要提供用于访问width和height的存取方法，使用这个类的程序也必须重写。应让客户端代码（使用你所编写代码的代码）能够以同样的方式对待所有的属性。

所幸Python能够替你隐藏存取方法，让所有的属性看起来都一样。通过存取方法定义的属性通常称为特性（property）。在Python中，实际上有两种创建特定的机制，我将重点介绍较新的那种——函数property，它只能用于新式类。随后，我将简单说明如何使用魔法方法来实现特性。

In [93]:
        class Rectangle:
            def __init__ (self):
              self.width = 0
              self.height = 0
            def set_size(self, size):
              self.width, self.height = size
            def get_size(self):
              return self.width, self.height
            size = property(get_size, set_size)  #实际size并不是类的属性，但可以通过property（）将其关联起来，直接
                #作为属性来访问

在这个新版的Rectangle中，通过调用函数property并将存取方法作为参数（获取方法在前，设置方法在后）创建了一个特性，然后将名称size关联到这个特性。这样，你就能以同样的方式对待width、height和size，而无需关心它们是如何实现的。

In [91]:
        >>> r = Rectangle()
        >>> r.width = 10
        >>> r.height = 5
        >>> r.size
      

(10, 5)

In [94]:

        >>> r.size = 150, 100
        >>> r.width


150

9.5.2 静态方法和类方法

在Python 2.4中，引入了一种名为装饰器的新语法，可用于像这样包装方法。（实际上，装饰器可用于包装任何可调用的对象，并且可用于方法和函数。）可指定一个或多个装饰器，为此可在方法（或函数）前面使用运算符@列出这些装饰器（指定了多个装饰器时，应用的顺序与列出的顺序相反）。

In [95]:
        class MyClass:
            @staticmethod
            def smeth():
              print('This is a static method')

            @classmethod
            def cmeth(cls):
              print('This is a class method of', cls)

In [96]:
        >>> MyClass.smeth()
      

This is a static method


In [97]:
        >>> MyClass.cmeth()
      

This is a class method of <class '__main__.MyClass'>


9.5.3 __getattr__、__setattr__等方法

可以拦截对对象属性的所有访问企图，其用途之一是在旧式类中实现特性（在旧式类中，函数property的行为可能不符合预期）。要在属性被访问时执行一段代码，必须使用一些魔法方法。下面的四个魔法方法提供了你需要的所有功能（在旧式类中，只需使用后面三个）。
❑ __getattribute__(self, name)：在属性被访问时自动调用（只适用于新式类）。
❑ __getattr__(self, name)：在属性被访问而对象没有这样的属性时自动调用。
❑ __setattr__(self, name, value)：试图给属性赋值时自动调用。
❑ __delattr__(self, name)：试图删除属性时自动调用。

相比函数property，这些魔法方法使用起来要棘手些（从某种程度上说，效率也更低），但它们很有用，因为你可在这些方法中编写处理多个特性的代码。然而，在可能的情况下，还是使用函数property吧。

In [99]:
        class Rectangle:
            def __init__ (self):
                self.width = 0
                self.height = 0
            def __setattr__(self, name, value):
                if name == 'size':
                    self.width, self.height = value
                else:
                    self. __dict__[name] = value
            def __getattr__(self, name):
                if name == 'size':
                    return self.width, self.height
                else:
                    raise AttributeError()

9.6 迭代器本书前面粗略地提及了迭代器（和可迭代对象），本节将更详细地介绍。对于魔法方法，这里只介绍__iter__，它是迭代器协议的基础。

In [101]:
        class Fibs:
            def __init__(self):
                self.a = 0
                self.b = 1
            def __next__(self):
                self.a, self.b = self.b, self.a + self.b
                return self.a
            def __iter__(self):
                return self

注意到这个迭代器实现了方法__iter__，而这个方法返回迭代器本身。在很多情况下，都在另一个对象中实现返回迭代器的方法__iter__，并在for循环中使用这个对象。但推荐在迭代器中也实现方法__iter__（并像刚才那样让它返回self），这样迭代器就可直接用于for循环中。

In [102]:
        >>> fibs = Fibs()

In [103]:
        >>> for f in fibs:
        ...      if f > 1000:
        ...          print(f)
        ...          break
        ...


1597


In [104]:
              >>> it = iter([1, 2, 3])
              >>> next(it)
              >>> next(it)


2

In [ ]:
 class TestIterator:
        value = 0
        def __next__(self):
            try:
                self.value += 1
                if self.value > 10:
                    raise StopIteration
                    return 0
                return self.value
            except Exception as e:
                print('except err: ',e)
                
        def __iter__(self):
             return self
 
       

In [ ]:
>>> ti = TestIterator()
>>> list(ti)

9.7 生成器生成器是一个相对较新的Python概念。由于历史原因，它也被称为简单生成器（simple generator）。生成器和迭代器可能是近年来引入的最强大的功能，但生成器是一个相当复杂的概念，你可能需要花些功夫才能明白其工作原理和用途。虽然生成器让你能够编写出非常优雅的代码，但请放心，无论编写什么程序，都完全可以不使用生成器。

In [1]:
        def flatten(nested):
            for sublist in nested:
              for element in sublist:
                  yield element

In [3]:
          >>> nested = [[1, 2], [3, 4], [5]]
          >>> for num in flatten(nested):
          ...      print(num)


1
2
3
4
5


In [4]:
          >>> list(flatten(nested))

[1, 2, 3, 4, 5]